In [1]:
import requests as req
from bs4 import BeautifulSoup as bs4
import pandas as pd
import numpy as np
import time
import random
import matplotlib.pyplot as plt
import seaborn as sns
import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementNotInteractableException
from webdriver_manager.chrome import ChromeDriverManager

from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LinearRegression as LinReg, Lasso, Ridge, ElasticNet
from sklearn.preprocessing import LabelEncoder, StandardScaler

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from fake_useragent import UserAgent


Limpiamos los Dataframes y los preparamos para el modelo

In [2]:
df_unido = pd.read_csv(r"C:\Users\Aleja\OneDrive\Documentos\PROYECTO2\df_unido.csv")
df_unido2 = pd.read_csv(r"C:\Users\Aleja\OneDrive\Documentos\PROYECTO2\df_unido2.csv")
df_unido3 = pd.read_csv(r"C:\Users\Aleja\OneDrive\Documentos\PROYECTO2\df_unido3.csv")
df_final = pd.read_csv(r"C:\Users\Aleja\OneDrive\Documentos\PROYECTO2\df_final.csv")

In [26]:
df_unido2.to_csv(r"C:\Users\Aleja\OneDrive\Documentos\PROYECTO2\df_unido2.csv",index=False)

In [7]:
df_unido

,Fecha,Equipo1,Equipo2
0,8th Finals,-,-
1,17 Feb 15,Paris SG,Chelsea
2,17 Feb 15,Shakhtar,Bayern Munich
3,18 Feb 15,Basel,FC Porto
4,18 Feb 15,Schalke 04,Real Madrid
...,...,...,...
259,10 May 23,AC Milan,Inter Milan
260,16 May 23,Inter Milan,AC Milan
261,17 May 23,Manchester City,Real Madrid
262,Final,-,-


In [8]:
df_unido = df_unido[df_unido['Equipo1'] != '-']

In [9]:
df_unido['Fecha'] = pd.to_datetime(df_unido['Fecha'])

C:\Users\Aleja\AppData\Local\Temp\ipykernel_2832\2550918867.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_unido['Fecha'] = pd.to_datetime(df_unido['Fecha'])
C:\Users\Aleja\AppData\Local\Temp\ipykernel_2832\2550918867.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unido['Fecha'] = pd.to_datetime(df_unido['Fecha'])


In [10]:
df_unido

,Fecha,Equipo1,Equipo2
1,2015-02-17,Paris SG,Chelsea
2,2015-02-17,Shakhtar,Bayern Munich
3,2015-02-18,Basel,FC Porto
4,2015-02-18,Schalke 04,Real Madrid
5,2015-02-24,Juventus,Dortmund
...,...,...,...
258,2023-05-09,Real Madrid,Manchester Cityty
259,2023-05-10,AC Milan,Inter Milan
260,2023-05-16,Inter Milan,AC Milan
261,2023-05-17,Manchester City,Real Madrid


In [11]:
relleno = None  # Variable para almacenar la fase actual
for i in df_unido.index:
    if df_unido.loc[i, 'Fecha'] == '8th Finals':
        relleno = '8th Finals'
    elif df_unido.loc[i, 'Fecha'] == 'Quarter-Finals':
        relleno = 'Quarter-Finals'
    elif df_unido.loc[i, 'Fecha'] == 'Semi-Finals':
        relleno = 'Semi-Finals'
    elif df_unido.loc[i, 'Fecha'] == 'Final':
        relleno = 'Final'
    else:
        df_unido.loc[i, 'Fase'] = relleno

C:\Users\Aleja\AppData\Local\Temp\ipykernel_2832\3074748117.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unido.loc[i, 'Fase'] = relleno


In [12]:
conteo1=df_unido['Equipo1'].value_counts()

In [13]:
conteo2 =df_unido['Equipo2'].value_counts()

In [14]:
conteo_general = conteo1 + conteo2
conteo_general

AC Milan              6.0
AS Roma               9.0
AS Roma  AET          NaN
Ajax Amsterd.         6.0
Ajax Amsterdam        2.0
Arsenal               6.0
Atalanta              2.0
Atletico Madrid      22.0
Atletico Madridid     NaN
Basel                 2.0
Bayern Munich        36.0
Benfica              14.0
Besiktas              2.0
Chelsea              21.0
Club Brugge           2.0
Dortmund             14.0
Dynamo Kiev           2.0
E. Frankfurt          2.0
FC Barcelona         27.0
FC Basel              2.0
FC Porto             18.0
FC Sevilla            8.0
Gent                  2.0
Inter Milan           9.0
Juventus             28.0
Lazio                 2.0
Leicester Ci.         4.0
Leverkusen            4.0
Lille                 2.0
Liverpool            27.0
Lyon                  2.0
Manchester City      30.0
Manchester Cityty     NaN
Manchester U.         8.0
Monaco               10.0
Monchengladbach       2.0
Napoli                6.0
PSV Eindhoven         2.0
Paris SG    

56 = es el numero de maxima aparaciones que puede aparecer un equipo en el torneo

In [11]:
conteo_general.dropna(inplace=True)

In [ ]:
conteo_general.sort_values(ascending=False)

In [ ]:
probabilidad = (conteo_general/56)*100
probabilidad

es el porcentaje de la cantidad maxima de partidos posibles que puede jugar a lo largos de los años

In [14]:
df_porcentaje = pd.DataFrame(probabilidad)

In [ ]:
probabilidad.sort_values(ascending=False)

In [20]:
df_porcentaje.rename(columns={'index': 'Equipos','count':'Porcentaje'},inplace=True)

In [22]:
df_porcentaje['Porcentaje'] = df_porcentaje['Porcentaje'].round(2)

In [30]:
df_porcentaje = df_porcentaje.sort_values(by='Porcentaje', ascending=False)

In [ ]:
df_porcentaje

In [15]:
equipos_reemplazo = {
    'Manchester C.': 'Manchester City',
    'Atletico Mad.': 'Atletico Madrid',
    'Manchester Utd': 'Manchester U.',
    'PSV Eindhoven.*': 'PSV Eindhoven',
    'Juventus.*AET': 'Juventus',
    'Bayern Munich.*AET': 'Bayern Munich',
    'ChelseaFeatured': 'Chelsea',
    'Chelsea.*AET': 'Chelsea',
    'Real MadridFeatured': 'Real Madrid',
    'FC Porto.*AET': 'FC Porto',
    'Manchester C.*AET': 'Manchester City',
    'Atletico Mad.*AETp.*': 'Atletico Madrid',
    'Paris SG.*AET': 'Paris SG',
    'Manchester Utd': 'Manchester U.',
    'Ajax Amsterd.': 'Ajax Amsterdam',
    'AS Roma  AET': 'AS Roma'
}

df_unido['Equipo1'] = df_unido['Equipo1'].replace(equipos_reemplazo)
df_unido['Equipo2'] = df_unido['Equipo2'].replace(equipos_reemplazo)


C:\Users\Aleja\AppData\Local\Temp\ipykernel_2832\3754494778.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unido['Equipo1'] = df_unido['Equipo1'].replace(equipos_reemplazo)
C:\Users\Aleja\AppData\Local\Temp\ipykernel_2832\3754494778.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unido['Equipo2'] = df_unido['Equipo2'].replace(equipos_reemplazo)


In [ ]:
equipo_correcciones =  {
    'Manchester C.': 'Manchester City',
    'Atletico Mad.': 'Atletico Madrid',
    'Manchester Utd': 'Manchester U.',
    'PSV Eindhoven.*': 'PSV Eindhoven',
    'Juventus.*AET': 'Juventus',
    'Bayern Munich.*AET': 'Bayern Munich',
    'ChelseaFeatured': 'Chelsea',
    'Chelsea.*AET': 'Chelsea',
    'Real MadridFeatured': 'Real Madrid',
    'FC Porto.*AET': 'FC Porto',
    'Manchester C.*AET': 'Manchester City',
    'Atletico Mad.*AETp.*': 'Atletico Madrid',
    'Paris SG.*AET': 'Paris SG',
    'Manchester Utd': 'Manchester U.',
    'Ajax Amsterd.': 'Ajax Amsterdam',
    'AS Roma  AET': 'AS Roma'
}
df_porcentaje['Equipos'] = df_porcentaje['Equipos'].replace(equipo_correcciones)

# Agrupar por el nombre del equipo y sumar los porcentajes
df_porcentaje = df_porcentaje.groupby('Equipos')['Porcentaje'].sum().reset_index()

In [33]:
df_porcentaje.to_csv(r"C:\Users\Aleja\OneDrive\Documentos\PROYECTO2\df_porcentaje.csv", index=False)

In [34]:
df_porcentaje =  pd.read_csv(r"C:\Users\Aleja\OneDrive\Documentos\PROYECTO2\df_porcentaje.csv")

In [49]:
df_porcentaje.head()

,Equipos,Porcentaje
0,Real Madrid,83.93
1,Manchester City,66.07
2,Bayern Munich,62.50
3,FC Barcelona,48.21
4,Liverpool,48.21


In [17]:
df_unido2

,Unnamed: 1,GP,W,D,L,GF,GA,GD,Pts,PPG,CS,FTS,Unnamed: 12
0,FC Barcelona,13,11,0,2,31,11,20,33,2.54,46%,0%,NaN
1,Real Madrid,12,8,2,2,24,9,15,26,2.17,58%,8%,NaN
2,Bayern Munich,12,8,1,3,33,13,20,25,2.08,50%,17%,NaN
3,Juventus,13,7,3,3,17,10,7,24,1.85,46%,31%,NaN
4,FC Porto,10,6,3,1,25,12,13,21,2.10,40%,0%,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,NaN,6,0,3,3,1,12,-11,3,0.50,33%,83%,FC Kobenhavn
252,NaN,6,1,0,5,7,21,-14,3,0.50,17%,17%,Maccabi Haifa
253,NaN,6,0,2,4,4,15,-11,2,0.33,0%,33%,Celtic
254,NaN,6,0,0,6,5,24,-19,0,0.00,0%,50%,Viktoria Plzen


In [18]:
df_unido3

,Matches of...,GP,Avg,0.5+,1.5+,2.5+,3.5+,4.5+,BTS,CS,FTS
0,BATE Borisov,6.0,4.33,100%,100%,83%,50%,50%,17%,0%,83%
1,Schalke 04,8.0,4.00,100%,88%,50%,50%,50%,62%,12%,25%
2,Sporting CP,6.0,4.00,100%,83%,67%,67%,33%,83%,0%,17%
3,Bayern Munich,12.0,3.83,92%,75%,67%,50%,42%,42%,50%,17%
4,Galatasaray,6.0,3.83,100%,100%,67%,67%,50%,67%,0%,33%
...,...,...,...,...,...,...,...,...,...,...,...
259,FC Kobenhavn,6.0,2.17,67%,67%,50%,17%,17%,17%,33%,83%
260,Chelsea,10.0,2.10,100%,80%,30%,0%,0%,30%,30%,40%
261,Leverkusen,6.0,2.00,83%,67%,33%,17%,0%,17%,33%,67%
262,Tottenham,8.0,1.88,75%,62%,25%,12%,12%,38%,38%,50%


In [19]:
df_final.head()

,H/A/T,Played,W,D,L,Equipos
0,Home,10,6,1,3,FC Kobenhavn
1,Away,10,6,2,2,FC Kobenhavn
2,Total,20,12,3,5,FC Kobenhavn
3,Home,12,8,2,2,RB Leipzig
4,Away,11,4,2,5,RB Leipzig


df_porcentaje

df_unido

df_unido2

df_unido3

df_final

In [ ]:
# Llenar los NaN en la columna "Unnamed: 1" con los valores de la columna "Unnamed: 12"
df_unido2['Unnamed: 1'].fillna(df_unido2['Unnamed: 12'], inplace=True)

# Eliminar la columna "Unnamed: 12" si ya no es necesaria
df_unido2.drop(columns=['Unnamed: 12'], inplace=True)

# Mostrar el DataFrame actualizado

In [24]:
df_unido2.rename(columns={'Unnamed: 1': 'Equipo'}, inplace=True)

In [4]:
df_unido2.drop(columns=['D','GF','GA'],inplace=True)

In [25]:
df_unido2

,Equipo,GP,W,D,L,GF,GA,GD,Pts,PPG,CS,FTS
0,FC Barcelona,13,11,0,2,31,11,20,33,2.54,46%,0%
1,Real Madrid,12,8,2,2,24,9,15,26,2.17,58%,8%
2,Bayern Munich,12,8,1,3,33,13,20,25,2.08,50%,17%
3,Juventus,13,7,3,3,17,10,7,24,1.85,46%,31%
4,FC Porto,10,6,3,1,25,12,13,21,2.10,40%,0%
...,...,...,...,...,...,...,...,...,...,...,...,...
251,FC Kobenhavn,6,0,3,3,1,12,-11,3,0.50,33%,83%
252,Maccabi Haifa,6,1,0,5,7,21,-14,3,0.50,17%,17%
253,Celtic,6,0,2,4,4,15,-11,2,0.33,0%,33%
254,Viktoria Plzen,6,0,0,6,5,24,-19,0,0.00,0%,50%
